### Passos Necessários

- Carregar o Modelo Final
    - Definir a base escolhida
    - Definir o algoritmo escolhido
    - Realizar o treinamento

- Escolher as bases que servirão de validação



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
import modelofuncoes as mf

# from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

importing Jupyter notebook from modelofuncoes.ipynb


### Definindo os algoritmos

In [2]:
hiperparametrosAB = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0, n_estimators=50, random_state=410)

hiperparametrosRF = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None, 
                                           criterion='gini', max_depth=None, max_features='auto', 
                                           max_leaf_nodes=None, max_samples=None, 
                                           min_impurity_decrease=0.0,
                                           min_samples_leaf=1, min_samples_split=2,
                                           min_weight_fraction_leaf=0.0, n_estimators=100,
                                           n_jobs=-1, oob_score=False, random_state=7374, verbose=0,
                                           warm_start=False)

hiperparametrosLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, 
                                       intercept_scaling=1, l1_ratio=None, max_iter=1000, multi_class='auto', 
                                       n_jobs=None, penalty='l2', random_state=1340, solver='lbfgs', 
                                       tol=0.0001, verbose=0, warm_start=False)
                    

hiperparametrosGB = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None,
                           random_state=2333, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

### Definindo a base de dados e Atributos Selecionados

In [16]:
#url = 'PreProcessamento/balanceadas/OC/ba-oc-6040-sp.xlsx'
url = "bases/Vacinas/OC/oc-vacina-6040-sp.xlsx"
alvo = 'evolucaoCaso'
baseObitoCurado = pd.read_excel(url)

#bases = {}
#bases['Base OC (60/40)'] = url

In [17]:
#atributosSelecionados = [ 'evolucaoCaso', 'dorDeGarganta', 'dispneia', 'coriza', 'dorDeCabeca', 
#                         'diabetes', 'obesidade', 'qntVacinas', 'faixaetaria', 'faixaDiasSintomas']

atributosSelecionados = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
                            'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca', 'evolucaoCaso']

baseObitoCurado = baseObitoCurado.loc[:, atributosSelecionados]

### Criando o modelo

In [18]:
# Separando as variáveis
x_train, x_test, y_train, y_test = train_test_split(baseObitoCurado.drop(alvo,axis=1),
                                                    baseObitoCurado[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [19]:
# Fit nos dados
modeloClassificador = hiperparametrosGB
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

### Multiplas Curva AUC

### Definindo as bases utilizadas e os atributos

In [20]:
listaBases = ["bases/Vacinas/OC/oc-vacina-6040-ac.xlsx", "bases/Vacinas/OC/oc-vacina-6040-al.xlsx", "bases/Vacinas/OC/oc-vacina-6040-am.xlsx", "bases/Vacinas/OC/oc-vacina-6040-ap.xlsx", "bases/Vacinas/OC/oc-vacina-6040-ba.xlsx", 
              "bases/Vacinas/OC/oc-vacina-6040-ce.xlsx", "bases/Vacinas/OC/oc-vacina-6040-df.xlsx", "bases/Vacinas/OC/oc-vacina-6040-go.xlsx", "bases/Vacinas/OC/oc-vacina-6040-ma.xlsx", "bases/Vacinas/OC/oc-vacina-6040-mg.xlsx", 
              "bases/Vacinas/OC/oc-vacina-6040-mt.xlsx", "bases/Vacinas/OC/oc-vacina-6040-pa.xlsx", "bases/Vacinas/OC/oc-vacina-6040-pb.xlsx", "bases/Vacinas/OC/oc-vacina-6040-pe.xlsx", "bases/Vacinas/OC/oc-vacina-6040-pi.xlsx", 
              "bases/Vacinas/OC/oc-vacina-6040-rj.xlsx", "bases/Vacinas/OC/oc-vacina-6040-rn.xlsx", "bases/Vacinas/OC/oc-vacina-6040-ro.xlsx", "bases/Vacinas/OC/oc-vacina-6040-rr.xlsx", "bases/Vacinas/OC/oc-vacina-6040-rs.xlsx"] 

colunasSelecionadasTeste = ['faixaetaria', 'dispneia', 'qntVacinas', 'dorDeGarganta', 
                            'coriza', 'diabetes', 'dorDeCabeca', 'cardiaca']

alvo = 'evolucaoCaso'

### Realizando a Validação

In [21]:
#for indice in range(0, len(listaBases)):

alvos = [0, 1]
desempenhoDosModelos = pd.DataFrame()

for base in listaBases:
    covidData = pd.read_excel(base)
    dadosTeste = covidData.loc[:, colunasSelecionadasTeste]
    dadosAlvo =  covidData.loc[:, alvo]
     
    metricasMap = mf.calcularMetricasValidacao(dadosTeste, dadosAlvo, base, modeloClassificador, alvos)
    desempenhoDosModelos = pd.concat([desempenhoDosModelos, pd.DataFrame([metricasMap])]) 

In [22]:
desempenhoDosModelos

,Acurácia,AUC,Base,Precisão (0),Recall (0),F1-Score (0),Precisão (1),Recall (1),F1-Score (1)
0,87.58,0.91,bases/Vacinas/OC/oc-vacina-6040-ac.xlsx,84.21,69.57,76.19,88.62,94.78,91.60
0,85.08,0.87,bases/Vacinas/OC/oc-vacina-6040-al.xlsx,79.84,63.98,71.03,86.64,93.53,89.95
0,77.64,0.80,bases/Vacinas/OC/oc-vacina-6040-am.xlsx,70.83,36.96,48.57,78.83,93.91,85.71
0,86.33,0.91,bases/Vacinas/OC/oc-vacina-6040-ap.xlsx,88.19,60.22,71.57,85.88,96.77,91.00
0,87.82,0.92,bases/Vacinas/OC/oc-vacina-6040-ba.xlsx,82.96,72.20,77.21,89.43,94.07,91.69
0,87.77,0.90,bases/Vacinas/OC/oc-vacina-6040-ce.xlsx,82.69,72.36,77.18,89.47,93.94,91.65
0,81.87,0.85,bases/Vacinas/OC/oc-vacina-6040-df.xlsx,75.00,55.10,63.53,83.70,92.62,87.94
0,82.88,0.80,bases/Vacinas/OC/oc-vacina-6040-go.xlsx,84.47,49.15,62.14,82.56,96.38,88.94
0,88.60,0.90,bases/Vacinas/OC/oc-vacina-6040-ma.xlsx,85.99,71.81,78.26,89.42,95.32,92.28
0,85.61,0.88,bases/Vacinas/OC/oc-vacina-6040-mg.xlsx,81.36,64.37,71.88,86.85,94.10,90.33
